## 一. Training Steps

### 1. the enviroment settings

In [1]:
gpu_id = 3
model_name = 'fair_single_best.pt'
ema_model_name = 'fair_ema_best.pt'

data_root = '../Data'

### 2. import some modules

In [2]:
import os
import sys
sys.path.append('../')
sys.path.append('/home/luxiongbo/data_bak/data/code/')

In [3]:
import random
import torch
import numpy as np
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from base_utils.dir_util import mkdirs
from tqdm import tqdm

### 3. some configs

In [4]:
import Config.config_mnist_rot as config
from Models.VIB_MNIST_ROT_whole_model import VariationalInformationBottleneck

In [5]:
from Utils.Get_Datasets import get_datasets
from Utils.Visualize import tsne_embedding_without_images
from Models.VIB_model import Weight_EMA

In [6]:
train_args = config.train_args()
dataset_args = config.dataset_args()
model_args = config.model_args()
use_cuda = True

### 4. fixed the seeds

In [7]:
train_args.gpu = gpu_id
init_seed = train_args.seed
torch.manual_seed(init_seed)
torch.cuda.manual_seed(init_seed)
torch.cuda.manual_seed_all(init_seed)
np.random.seed(init_seed)
random.seed(init_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
os.environ["CUDA_VISIBLE_DEVICES"] = str(train_args.gpu)

### 5. loading datasets...

In [8]:
train_loader, test_loader, test_55_loader, test_65_loader = get_datasets(
    dataset_args.dataset, dataset_args.train_batch_size, dataset_args.test_batch_size, root=data_root)

Loading mnist-rot dataset...
Done!



### 6. print some key vaules

In [9]:
parameter = {
    "reconstruction_weight": model_args.reconstruction_weight,
    "pairwise_kl_clean_weight": model_args.pairwise_kl_clean_weight,
    "pairwise_kl_noise_weight": model_args.pairwise_kl_noise_weight,
    "sparse_kl_weight_clean": model_args.sparse_kl_weight_clean,
    "sparse_kl_weight_noise": model_args.sparse_kl_weight_noise,
    "sparsity_clean": model_args.sparsity_clean,
    "sparsity_noise": model_args.sparsity_noise,
    "num_sensitive_class": dataset_args.num_sensitive_class
}
for k, v in parameter.items():
    print(k, ':\t',v)

reconstruction_weight :	 0.01
pairwise_kl_clean_weight :	 0.075
pairwise_kl_noise_weight :	 0.01
sparse_kl_weight_clean :	 0.1
sparse_kl_weight_noise :	 0.01
sparsity_clean :	 0.1
sparsity_noise :	 0.1
num_sensitive_class :	 5


### 7. create the saved dir...

In [10]:
path_1 = '%.03f_%.03f_%.03f' %(model_args.reconstruction_weight, model_args.pairwise_kl_clean_weight, model_args.pairwise_kl_noise_weight)
path_2 = '%.02f_%.02f_%.02f_%.02f' %(model_args.sparse_kl_weight_clean, model_args.sparse_kl_weight_noise, model_args.sparsity_clean, model_args.sparsity_noise)
save_model_dir = os.path.join('../saved_model/Mnist-Rot/', path_1, path_2, str(train_args.seed))
mkdirs(save_model_dir)
print(save_model_dir)

../saved_model/Mnist-Rot/0.010_0.075_0.010/0.10_0.01_0.10_0.10/0


### 8. define the model

In [11]:
vib_model = VariationalInformationBottleneck(
    dataset_args.shape_data, dataset_args.num_target_class,
    model_args.dim_embedding_clean, model_args.dim_embedding_noise,
    model_args.channel_hidden_encoder, model_args.channel_hidden_decoder,
    model_args.dim_hidden_classifier, parameter
)

vib_model_copy = VariationalInformationBottleneck(
    dataset_args.shape_data, dataset_args.num_target_class,
    model_args.dim_embedding_clean, model_args.dim_embedding_noise,
    model_args.channel_hidden_encoder, model_args.channel_hidden_decoder,
    model_args.dim_hidden_classifier, parameter
)

if use_cuda:
    vib_model = vib_model.cuda()
    vib_model_copy = vib_model_copy.cuda()

### 9. optimizer

In [12]:
vib_model_EMA = Weight_EMA(vib_model_copy, vib_model.state_dict(), decay=0.999)
optimizer = torch.optim.Adam(vib_model.parameters(), lr=train_args.lr, betas=(0.9, 0.999), weight_decay=1e-4)
lr_scheduler = StepLR(optimizer, step_size=2, gamma=0.95)

### 10. train one epoch

In [13]:
def train_one_epoch():
    total_loss_total = 0.0
    classification_loss_total = 0.0
    classification_sensitive_loss_total = 0.0
    reconstruction_loss_total = 0.0
    pairwise_kl_loss_clean_total = 0.0
    pairwise_kl_loss_noise_total = 0.0
    sparse_kl_loss_clean_total = 0.0
    sparse_kl_loss_noise_total = 0.0

    for iter_index, (images, labels, sensitive_labels) in enumerate(train_loader):
        images = Variable(images.unsqueeze(dim=1).float())
        labels = Variable(labels.long())
        sensitive_labels = Variable(sensitive_labels.long())
        if use_cuda:
            images = images.cuda()
            labels = labels.cuda()
            sensitive_labels = sensitive_labels.cuda()
        
        (total_loss, classification_loss, classification_sensitive_loss, reconstruction_loss, pairwise_kl_loss_clean,
         pairwise_kl_loss_noise, sparse_kl_loss_clean, sparse_kl_loss_noise) = vib_model(
            images, labels, input_sensitive_labels=sensitive_labels, num_samples=10, training=True
        )
        
        total_loss_total = total_loss_total + total_loss.sum(-1)
        classification_loss_total = classification_loss_total + classification_loss.sum(-1)
        classification_sensitive_loss_total = classification_sensitive_loss_total + classification_sensitive_loss.sum(-1)
        reconstruction_loss_total = reconstruction_loss_total + reconstruction_loss.sum(-1)
        pairwise_kl_loss_clean_total = pairwise_kl_loss_clean_total + pairwise_kl_loss_clean.sum(-1)
        pairwise_kl_loss_noise_total = pairwise_kl_loss_noise_total + pairwise_kl_loss_noise.sum(-1)
        sparse_kl_loss_clean_total = sparse_kl_loss_clean_total + sparse_kl_loss_clean.sum(-1)
        sparse_kl_loss_noise_total = sparse_kl_loss_noise_total + sparse_kl_loss_noise.sum(-1)

        optimizer.zero_grad()
        total_loss.mean(-1).backward()
        optimizer.step()
        vib_model_EMA.update(vib_model.state_dict())
    lr_scheduler.step()
    total_loss_mean = total_loss_total / len(train_loader.dataset)
    classification_loss_mean = classification_loss_total / len(train_loader.dataset)
    classification_sensitive_loss_mean = classification_sensitive_loss_total / len(train_loader.dataset)
    reconstruction_loss_mean = reconstruction_loss_total / len(train_loader.dataset)
    pairwise_kl_loss_clean_mean = pairwise_kl_loss_clean_total / len(train_loader.dataset)
    pairwise_kl_loss_noise_mean = pairwise_kl_loss_noise_total / len(train_loader.dataset)
    sparse_kl_loss_clean_mean = sparse_kl_loss_clean_total / len(train_loader.dataset)
    sparse_kl_loss_noise_mean = sparse_kl_loss_noise_total / len(train_loader.dataset)
    return (total_loss_mean, classification_loss_mean, classification_sensitive_loss_mean,
            reconstruction_loss_mean, pairwise_kl_loss_clean_mean, pairwise_kl_loss_noise_mean,
            sparse_kl_loss_clean_mean, sparse_kl_loss_noise_mean)

### 11. evaluation

In [14]:
def evaluation(epoch_index, test_dataloader, is_drawing=False):
    vib_model.eval()
    vib_model_EMA.model.eval()
    avg_correct = 0.0
    single_correct = 0.0

    valid_embedding_labels = []
    valid_embedding_sensitive_labels = []
    valid_embedding_clean_images = []

    for iter_index, data in enumerate(test_dataloader):
        images = data[0]
        labels = data[1]
        images = Variable(images.unsqueeze(dim=1).float())
        labels = Variable(labels.long())
        if use_cuda:
            images = images.cuda()
            labels = labels.cuda()

        avg_embedding, _, avg_classification_prob = vib_model_EMA.model(
            images, labels, num_samples=100, training=False
        )
        avg_prediction = avg_classification_prob.max(1)[1]
        avg_correct = avg_correct + torch.eq(avg_prediction, labels).float().sum()

        single_embedding, _, single_classification_prob = vib_model(
            images, labels, num_samples=100, training=False
        )
        single_prediction = single_classification_prob.max(1)[1]
        single_correct = single_correct + torch.eq(single_prediction, labels).float().sum()

        if is_drawing:
            valid_embedding_labels.extend(np.asarray(labels.detach().numpy()))
            valid_embedding_clean_images.extend(np.asarray(single_embedding.detach().numpy()))

    if is_drawing:
        tsne_embedding_without_images(images=valid_embedding_clean_images,
                                      labels=[valid_embedding_sensitive_labels],
                                      save_name="../Log/result_" + str(epoch_index) + "_clean.png")

    avg_correct_mean = avg_correct / len(test_dataloader.dataset)
    single_correct_mean = single_correct / len(test_dataloader.dataset)
    return avg_correct_mean * 100, single_correct_mean * 100

### 12. training code

In [15]:
def train():
    best_avg_correct = 0.0
    best_single_correct = 0.0
    
    epoches = 500  # train_args.max_epoch
    for epoch_index in tqdm(range(epoches)):
        vib_model.train()
        vib_model_EMA.model.train()
        
        # avg_correct, single_correct = evaluation(epoch_index + 1, test_loader)
        
        (total_loss, classification_loss, classification_sensitive_loss, reconstruction_loss,
         pairwise_kl_loss_clean, pairwise_kl_loss_noise, sparse_kl_loss_clean, sparse_kl_loss_noise) = train_one_epoch()
        if (epoch_index + 1) % 2 == 0:
            print('[train]Epoch: {}, total_loss: {:.4}, classification_loss: {:.4}, classification_sensitive_loss: {:.4}, '
                  'reconstruction_loss: {:.4}, pairwise_kl_loss_clean: {:.4}, pairwise_kl_loss_noise: {:.4}, '
                  'sparse_kl_loss_clean: {:.4}, sparse_kl_loss_noise: {:.4}'
                  .format(epoch_index + 1, total_loss, classification_loss, classification_sensitive_loss, reconstruction_loss,
                          pairwise_kl_loss_clean, pairwise_kl_loss_noise, sparse_kl_loss_clean, sparse_kl_loss_noise))

        if (epoch_index + 1) % 2 == 0:
            is_drawing = False
            print('##################### test #####################')
            avg_correct, single_correct = evaluation(epoch_index + 1, test_loader, is_drawing=is_drawing)
        
            if best_avg_correct <= avg_correct:
                best_avg_correct = avg_correct
                print("##################### save #####################")
                torch.save(vib_model_EMA.model.state_dict(), 
                           os.path.join(save_model_dir, ema_model_name))

            if best_single_correct <= single_correct:
                best_single_correct = single_correct
                print("##################### save #####################")
                torch.save(vib_model.state_dict(),
                           os.path.join(save_model_dir, model_name))

            print('[test]Epoch: {}, avg_correct: {:.4}, best_avg_correct: {:.4}, '
                  'single_correct: {:.4}, best_single_correct: {:.4}'
                  .format(epoch_index + 1, avg_correct, best_avg_correct,
                          single_correct, best_single_correct))

### 13. start training

In [ ]:
train()

  0%|▎                                                                                                                                                                 | 1/500 [03:58<33:06:52, 238.90s/it]